# Test utils

## Import Data

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import sys
import string
import re
import nltk
import utils.parse_xml as ParseXML


print(sys.version)
print(sys.executable)

# Specify the path to your XML file
#xml_file_path = r'..\data\test_data\sample.xml'
# Define the namespacetree
#namespace = {}

# Specify the path to your XML file
xml_file_path = r'..\data\PURE\requirements-xml\0000 - cctns.xml'
# Define the namespacetree
namespace = {'ns': 'req_document.xsd'}

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

df = ParseXML.parse_xml_to_dataframe(xml_file_path, namespace)

display(df.head(10))

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\anaconda3\python.exe


,tag,text,id,path
0,req_document,,,/req_document
1,title,,,req_document/title
2,title,E-GOVERNANCE MISSION MODE PROJECT (MMP),,req_document/title/title
3,title,CRIME & CRIMINAL TRACKING NETWORK AND SYSTEMS ...,,req_document/title/title
4,title,FUNCTIONAL REQUIREMENTS SPECIFICATION V1.0 (DR...,,req_document/title/title
5,title,MINISTRY OF HOME AFFAIRS GOVERNMENT OF INDIA,,req_document/title/title
6,version,1.0,,req_document/version
7,p,,1,req_document/p
8,title,INTRODUCTION,,req_document/p/title
9,text_body,The Functional Requirements Specifications (FR...,,req_document/p/text_body


In [4]:
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer


stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
lemmatizer = WordNetLemmatizer()


def remove_punct(text):
    # FIXME: the '-' caracter is deleted
    # FIXME: multiple whitespaces are not deleted
    text_nopunct = "".join(
        [char for char in text if char not in string.punctuation])
    return text_nopunct


def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens


def lemmatizing(tokenized_text):
    # Initialize WordNetLemmatizer
    # Lemmatize each word in the tokenized text
    text = [lemmatizer.lemmatize(word) for word in tokenized_text]

    return text


def clean_text_original(text):
    text = "".join([word.lower()
                   for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word)
                    for word in tokens if word not in stopwords])
    return text


def clean_text(text, remove_punct=True, remove_stopwords=True):
    """
    Clean the input text by removing punctuation, stopwords, and performing tokenization.

    Parameters:
    - text (str): The input text to be cleaned.
    - remove_punct (bool): Flag to indicate whether to remove punctuation or not. Default is True.
    - remove_stopwords (bool): Flag to indicate whether to remove stopwords or not. Default is True.

    Returns:
    - cleaned_text (list of str): The cleaned list of tokens.
    """

    # FIXME:  V1.0 tokenized as "v1","0"

    # Remove punctuation
    if remove_punct:
        text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text using regular expression
    tokens = re.findall(r'\b\w+(?:-\w+)*\b', text.lower())

    # Remove stopwords if required
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

    return tokens


def join_list_to_string(attribute):
    if isinstance(attribute, list):
        return ' '.join(attribute)
    else:
        return str(attribute)


def clean_and_lemm_text(text):
    text_clean = clean_text(text, False, False)
    text_lemm = lemmatizing(text_clean)
    list_2_word = join_list_to_string(text_lemm)

    return list_2_word


In [6]:
df['clean_text'] = df['text'].apply(lambda x: clean_text(x.lower(),False,False))
df['lemmatizing'] = df['clean_text'].apply(lambda x: lemmatizing(x))
df['join_list_to_string'] = df['lemmatizing'].apply(lambda x: join_list_to_string(x))

df.head()

,tag,text,id,path,clean_text,lemmatizing,join_list_to_string
0,req_document,,,/req_document,[],[],
1,title,,,req_document/title,[],[],
2,title,E-GOVERNANCE MISSION MODE PROJECT (MMP),,req_document/title/title,"[e-governance, mission, mode, project, mmp]","[e-governance, mission, mode, project, mmp]",e-governance mission mode project mmp
3,title,CRIME & CRIMINAL TRACKING NETWORK AND SYSTEMS ...,,req_document/title/title,"[crime, criminal, tracking, network, and, syst...","[crime, criminal, tracking, network, and, syst...",crime criminal tracking network and system cctns
4,title,FUNCTIONAL REQUIREMENTS SPECIFICATION V1.0 (DR...,,req_document/title/title,"[functional, requirements, specification, v1, ...","[functional, requirement, specification, v1, 0...",functional requirement specification v1 0 draft


In [8]:
df['clean_and_lemm_text'] = df['text'].apply(lambda x: clean_and_lemm_text(x))
df.head()

,tag,text,id,path,clean_text,lemmatizing,join_list_to_string,clean_and_lemm_text
0,req_document,,,/req_document,[],[],,
1,title,,,req_document/title,[],[],,
2,title,E-GOVERNANCE MISSION MODE PROJECT (MMP),,req_document/title/title,"[e-governance, mission, mode, project, mmp]","[e-governance, mission, mode, project, mmp]",e-governance mission mode project mmp,e-governance mission mode project mmp
3,title,CRIME & CRIMINAL TRACKING NETWORK AND SYSTEMS ...,,req_document/title/title,"[crime, criminal, tracking, network, and, syst...","[crime, criminal, tracking, network, and, syst...",crime criminal tracking network and system cctns,crime criminal tracking network and system cctns
4,title,FUNCTIONAL REQUIREMENTS SPECIFICATION V1.0 (DR...,,req_document/title/title,"[functional, requirements, specification, v1, ...","[functional, requirement, specification, v1, 0...",functional requirement specification v1 0 draft,functional requirement specification v1 0 draft
